# Homework 5
## 1. Around the semicircular law
### 1

In [57]:
import numpy as np
from scipy import stats
np.random.seed(10)
def wigner_density(x):
    u = 4-x**2
    if u > 0:
        u = np.sqrt(u)
    else:
        u = 0
    return u/(np.pi*2)
wigner_density_vectorized = np.vectorize(wigner_density)

### 2

In [58]:
def generate_wigner(n):
    if type(n) is not int:
        raise TypeError("n os not an integer")
    elif n<= 0:
        raise ValueError("n is not positive")
    else:
        a = np.reshape(np.random.normal(0,1/np.sqrt(n),n**2),(n,n))
        for i in range(1,n):
            for j in range(i):
                a[j,i] = a[i,j]
        return a

### 3

In [59]:
def get_spectrum(a):
    b, d = np.linalg.eigh(a)
    return b

### 4

In [60]:
import matplotlib as mp
import matplotlib.pyplot as plt
x = np.arange(-2,2.01,0.01)
d = wigner_density_vectorized(x)



In [ ]:
fig, axs = plt.subplots(4, 1, figsize=(10,20 ), sharey=True)
nlist = [100,200,500,1000]
for i in range(4):
    n = nlist[i]
    eigen = get_spectrum(generate_wigner(n))
    nbin = np.int(np.sqrt(n))
    axs[i].hist(eigen,bins = nbin ,density = 1)
    axs[i].plot(x,d,"r")
    axs[i].set_title("$n = %s$" % str(n))
    axs[i].set_ylabel("Density")
    axs[i].set_xlabel("Eigen Value")
fig.suptitle("Histograms")  
plt.subplots_adjust(hspace = 0.3)
    


Since, the empirical distribution of the eigen values of Wigner matrix converges weakly almost-surely to the semi-circular distribution, we calculate the approximate proportion (calculated using 500 iterations) of time we are rejecting the null hypothesis that empirical spectral distribution is Wigner distribtion in KS test. We look for the $n$ for which is proportion is less than 0.001 which happen to be 10. So, 45 is a reasonable choice. In KS test we are setting a very high level of significance (0.99) to have a high power in test. 

In [ ]:
from scipy import stats as st
n = 0
error = 1
while error > 1e-2:
    n += 2
    decision = np.array([0]*500)
    for i in range(500):
        ev = get_spectrum(generate_wigner(n))
        D, q = st.kstest(ev/2, "semicircular",alternative="two-sided")
        if q < 0.99:
            decision[i] = 1
    error = np.mean(decision)
    print("Type 1 error for n = ",n,"is ",error)

## 2. Plotting a Mixture of Normals
### 1

In [ ]:
lev = np.load("labels.npy")
pt = np.loadtxt("points.dlm",delimiter = "\t")

In [ ]:
col = ['r']*len(lev)
for i in range(len(lev)):
    if lev[i] == 0:
        col[i] = "b"

### 2

In [ ]:
plt.scatter(pt[:,0],pt[:,1],c = col)
plt.xlabel("$x$")
plt.ylabel("$y$")
plt.xlim((0,1))
plt.ylim((0,1))

### 3 and 4

In [ ]:
from scipy import stats as st
mu0 = np.array([0.2,0.7])
mu1 = np.array([0.65,0.3])
sigma0 = np.array([[0.015,-0.011],[-0.011,0.018]])
sigma1 = np.array([[0.016,-0.011],[-0.011,0.016]])
det0 = np.linalg.det(sigma0)
det1 = np.linalg.det(sigma1)
r = st.chi2(2)
l = np.exp(-r.ppf((0.95,0.68))/2)/(2*np.pi)


In [ ]:
level0 = l/np.sqrt(det0)
level1 = l/np.sqrt(det1)
mvn0 = st.multivariate_normal(mu0,sigma0)
mvn1 = st.multivariate_normal(mu1,sigma1)

x, y = np.mgrid[0:1:.01,0:1:.01]
pos = np.empty(x.shape + (2,))
pos[:,:,0] = x; pos[:,:,1] = y

plt.scatter(pt[:,0],pt[:,1],c = col)
plt.xlabel("$x$")
plt.ylabel("$y$")
plt.xlim((0,1))
plt.ylim((0,1))

plt.contour(x,y,mvn0.pdf(pos),levels = level0, colors = 'b')
plt.contour(x,y,mvn1.pdf(pos),levels = level1, colors = 'r')
ind = np.argmax(pt[:,0])
plt.annotate('outlier', xy = pt[ind], xytext = (1.1,0.3), fontsize = 14, arrowprops = dict(facecolor = 'black', shrink = 0.02))

There is an obvious outlier in the data annotted above. Other than that the data looks to come from multivariate normal.

## 3. Conway's Game of Life
### 1

In [ ]:
def is_valid_board(a):
    dim = a.shape
    if len(dim) != 2:
        return False
    else:
        b = a == 0.0
        c = a == 1.0
        d = b + c
        return d.all()

### 2

In [ ]:
def count_neighbour(a, ind = (0,0)):
    dim = a.shape
    live  = 0
    if is_valid_board(a) == False:
        raise ValueError("Not a valid board for game of life")
    else:
        x = np.mgrid[-1:2:1,-1:2:1]
        y = []
        for i in range(3):
            for j in range(3):
                if (i,j) != (1,1):
                    y.append(x[:,i,j])
        y = np.array(y,dtype = "int") + ind
        for i in y:
            if i[0] < 0:
                i[0] = dim[0] - 1
            if i[1] < 0:
                i[1] = dim[1] - 1
            if i[0] > dim[0]-1:
                i[0] = 0
            if i[1] > dim[1]-1:
                i[1] = 0
            live += a[i[0],i[1]]
        return live


In [ ]:
def gol_step(a):
    if is_valid_board(a) == False:
        raise ValueError("Not a valid board for game of life")
    b = a.copy()
    dim = a.shape
    for i in range(dim[0]):
        for j in range(dim[1]):
            live = count_neighbour(a,ind = (i,j))
            if a[i,j] == 0:
                if live == 3:
                    b[i,j] = 1
            if a[i,j] == 1:
                if (live < 2 or live > 3):
                    b[i,j] = 0
    return b
                
                


### 3

In [ ]:
def draw_gol_board(a):
    if is_valid_board(a) == False:
        raise ValueError("Not a valid board for game of life")
    img = plt.imshow(a, cmap = "Greys")
    plt.xticks([],[])
    plt.yticks([],[])
    return img

### 4

In [ ]:
a = np.zeros(shape = (20,20))
a[1,2] = 1
a[2,3] = 1
a[3,1:4:1] = 1

draw_gol_board(a)

### 5

In [ ]:
a = np.zeros(shape = (20,20))
a[1,2] = 1
a[2,3] = 1
a[3,1:4:1] = 1

plt.subplot(151)
draw_gol_board(a)
plt.title("$t = 0$")
plt.subplot(152)
a = gol_step(a)
draw_gol_board(a)
plt.title("$t = 1$")
plt.subplot(153)
a = gol_step(a)
draw_gol_board(a)
plt.title("$t = 2$")
plt.subplot(154)
a = gol_step(a)
draw_gol_board(a)
plt.title("$t = 3$")
plt.subplot(155)
a = gol_step(a)
draw_gol_board(a)
plt.title("$t = 4$")
_ = plt.tight_layout()